---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **`Caching`**

* LangChain: LLM을 위한 `선택적 캐싱 레이어`를 제공

### **`Caching` 유용성**

* **동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 `API 호출 횟수를 줄여 비용 절감`**

* **LLM 제공업체에 대한 API 호출 횟수를 줄여 `애플리케이션의 속도 증가`**

In [ ]:
# 환경변수 처리 및 클라어트 생성
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
import os
import json

# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
# LangSmith 추적 설정하기 (https:smith.langchin.com)
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable                                                             # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음"      # API 키 값은 직접 출력하지 않음
org = "설정됨" if os.getenv('LANGCHAIN_ORGANIZATION') else "설정되지 않음"                     # 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")


<small>

* 셀 출력

    ```markdown
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

---

### **`모델`, `프롬프트`** 생성하기

* **`모델` 생성**

In [9]:
# LLM 초기화
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        #temperature=0.7,                                    
        max_output_tokens=4096,
    )

* **`프롬프트` 생성**

In [10]:
# 프롬프트 생성

prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

* **`체인` 생성**

In [11]:
# 체인 생성

chain = prompt | gemini_lc

In [ ]:
%%time 

response = chain.invoke({"country": "한국"})
print(response.content)

<small>

* 셀 출력 (1.6s)

    ```markdown
    한국은 동아시아에 위치한 반도 국가로, 풍부한 역사와 독특한 문화를 자랑합니다. K팝, 드라마 등 한류 문화는 전 세계적으로 큰 인기를 얻고 있으며, IT 기술 강국으로서 혁신적인 발전을 이루고 있습니다. 아름다운 자연 경관과 더불어 활기찬 도시 생활을 동시에 경험할 수 있는 매력적인 나라입니다.
    CPU times: user 61.4 ms, sys: 62.7 ms, total: 124 ms
    Wall time: 1.62

---

### **`InMemoryCache`**

* `인메모리 캐시`
  * `동일 질문`에 대한 `답변을 저장`
  * 캐시에 `저장된 답변을 반환`

In [ ]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시 사용하기
set_llm_cache(InMemoryCache())

# 체인 실행
response = chain.invoke({"country": "한국"})
print(response.content)

<small>

* 셀 출력 (2.65s)

    ```markdown
    한국은 동아시아에 위치한 반도 국가로, 풍부한 역사와 전통을 자랑합니다. IT 강국으로 알려져 있으며, K-팝, K-드라마 등 한류 문화는 전 세계적으로 큰 인기를 얻고 있습니다. 

    정치적으로는 민주주의 공화국이며, 경제적으로는 세계 10위권의 경제 대국입니다. 아름다운 자연경관과 맛있는 음식 또한 한국을 매력적으로 만드는 요소입니다. 200자 내외로 간략하게 요약했습니다.
    CPU times: user 154 ms, sys: 55 ms, total: 209 ms
    Wall time: 2.65 s
    ```

In [ ]:
%%time
    
# 체인 실행하기
response = chain.invoke({"country": "한국"})
print(response.content)

<small>

* 셀 출력 (0.0s) (20.7ms)

    ```markdown
    한국은 동아시아에 위치한 반도 국가로, 풍부한 역사와 전통을 자랑합니다. IT 강국으로 알려져 있으며, K-팝, K-드라마 등 한류 문화는 전 세계적으로 큰 인기를 얻고 있습니다. 

    정치적으로는 민주주의 공화국이며, 경제적으로는 세계 10위권의 경제 대국입니다. 아름다운 자연경관과 맛있는 음식 또한 한국을 매력적으로 만드는 요소입니다. 200자 내외로 간략하게 요약했습니다.
    CPU times: user 6.09 ms, sys: 13.5 ms, total: 19.6 ms
    Wall time: 20.7 ms
    ```

---

### **`SQLite Cache`**

In [15]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리 생성하기
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache 사용해보기
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [ ]:
%%time 

# 체인 실행하기
response = chain.invoke({"country": "한국"})
print(response.content)

<small>

* 셀 출력 (1.0s)

    ```markdown
    한국은 동아시아에 위치한 반도 국가로, 풍부한 역사와 독특한 문화를 자랑합니다. IT 강국으로서 첨단 기술 발전을 선도하고 있으며, K-팝, K-드라마 등 한류 문화는 전 세계적으로 큰 인기를 얻고 있습니다. 맛있는 음식과 아름다운 자연, 그리고 친절한 사람들 덕분에 많은 관광객들이 찾는 매력적인 나라입니다.
    CPU times: user 25.7 ms, sys: 19.4 ms, total: 45.1 ms
    Wall time: 1.07 s
    ```

---

### **정리**

* **`InMemoryCache`**
  * **빠르고 일시적인 저장**에 유리
  * 주로 **단기적인 데이터**를 빠르게 처리할 때 사용
  
* **`SQLite Cache`**
  * **영구적인 데이터 저장**에 유리
  * 컴퓨터를 꺼도 데이터가 남기 때문에 **장기적인 저장**이 필요할 때 사용

<small>

* **`InMemoryCache`값과 `SQLite Cache`결과값이 다른 이유**

  | **단계**               | **SQLite 캐시 사용 (첫 번째 답 생성)**            | **InMemoryCache 사용 (두 번째 답 생성)**    |
  |----------------------|-----------------------------------------------|----------------------------------------|
  | **답 생성**           | 첫 번째 답 생성 후 **SQLite 캐시 사용**        | 두 번째 답 생성 후 **InMemoryCache 사용** |
  | **저장 위치**          | **디스크**에 저장                            | **RAM**에 저장                         |
  | **데이터 유지**        | 컴퓨터를 껐다 켜도 **유지됨**                   | 컴퓨터를 껐다 켜면 **사라짐**             |
  | **데이터 접근 속도**   | 디스크에서 불러오는 속도 (상대적으로 느림)      | 메모리에서 빠르게 불러오는 속도         |
  | **결과**               | 동일한 체인으로 답을 불러와 **같은 답**이 나옴   | 빠르게 캐시된 데이터를 불러와 **같은 답**이 나옴  |

<br>

  ***

<br>

* **현상 설명 순서도 (Flowchart)**

  * 실행 순서: 첫 체인 생성 후 답변 생성 (첫번째 답변 생성) → `InMemoryCache` 사용 → 두번째 답변 생성(`다른 답변`) → `RAM` 저장 → `재출력` → `두번째 답변` 출력 → `SQLite Cache` 사용 → `디스크`에 저장된 답변 가져옴 = 첫번째 답변 접근 → `첫번째 답변` 출력

* ![설명 순서도](../04_Model/images/caching.png)


---

### **비교표**

  | 특징         | **`InMemoryCache` (메모리 캐시)** | **`SQLite Cache` (SQLite 캐시)**  |
  | ---------- | -------------------------- | ----------------------------- |
  | **저장 위치**  | 컴퓨터의 **`RAM`(메모리)** 에 저장  | **디스크 파일** 에 저장                |
  | **속도**     | 매우 빠름                      | 메모리보다는 느림                     |
  | **데이터 유지** | 컴퓨터를 끄면 데이터가 **사라짐**       | 컴퓨터를 꺼도 데이터가 **남음**           |
  | **장점**     | 매우 빠르고 효율적                 | 데이터를 오랫동안 저장 가능, 데이터가 사라지지 않음 |
  | **단점**     | 컴퓨터를 껐을 때 **모든 데이터가 사라짐**  | 빠르지 않아서 읽기/쓰기 `속도가 느릴 수 있음`     |

---

* *next: 모델 직렬화(Serialization) - 저장 및 불러오기*

---